In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Helpful
import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *
from src.utils.data_split import *
from src.models.EMOS import *
import data.raw.load_data_raw as ldr
import data.processed.load_data_processed as ldp
import data.processed.load_data_processed_denormed as ldpd
from src.models.CRPS_baseline.CRPS_load import *

2023-05-28 14:42:00.274943: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-05-28 14:42:00.274962: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<IPython.core.display.Javascript object>

### 1. Load Raw Data

In [13]:
# Load train data for variables t2m and ws10
train_t2m_ws10_denormed = ldpd.load_data_t2m_ws10_train_proc_denorm()
train_t2m_denormed = train_t2m_ws10_denormed[0]
train_ws10_denormed = train_t2m_ws10_denormed[1]

# Load test data for variables t2m and ws10
test_t2m_ws10_denormed = ldpd.load_data_t2m_ws10_test_proc_denorm()
test_ws10_denormed = test_t2m_ws10_denormed[0]
test = test_t2m_w10_denormed[1]

<IPython.core.display.Javascript object>

###  2. Data Split

In [15]:
# Train-test split into lead_times for t2m
X_train_t2m_denormed, y_train_t2m_denormed = split_var_lead_one(train_t2m_denormed)
X_test_t2m_denormed, y_test_t2m_denormed = split_var_lead_one(test_t2m_denormed)
# Train-test split into lead_times for ws10
X_train_ws10_denormed, y_train_ws10_denormed = split_var_lead_one(train_ws10_denormed)
X_test_ws10_denormed, y_test_ws10_denormed = split_var_lead_one(test_ws10_denormed)

<IPython.core.display.Javascript object>

### 3. Train EMOS model

In [123]:
EMOS_model = build_EMOS_network_keras(compile=True, lr=0.01)

<IPython.core.display.Javascript object>

In [124]:
EMOS_model.fit(
    [
        X_train_t2m_denormed[1].isel(mean_std=0).values.flatten(),
        X_train_t2m_denormed[1].isel(mean_std=1).values.flatten(),
    ],
    y_train_t2m_denormed[1].values.flatten(),
    epochs=5,
    batch_size=4096,
)

Epoch 1/5
5443/5443 [==============================] - 12s 2ms/step - loss: 397.8957
Epoch 2/5
5443/5443 [==============================] - 11s 2ms/step - loss: 367.9882
Epoch 3/5
5443/5443 [==============================] - 11s 2ms/step - loss: 336.8665
Epoch 4/5
5443/5443 [==============================] - 11s 2ms/step - loss: 299.5947
Epoch 5/5
5443/5443 [==============================] - 11s 2ms/step - loss: 257.7518


<IPython.core.display.Javascript object>

In [128]:
t2m_crps_baseline = crps_load_lead_lat_lon("t2m")

<IPython.core.display.Javascript object>

In [142]:
t2m_crps_baseline[0]

array([[0.72393841, 0.71896124, 0.72366452, ..., 1.58918556, 1.52636751,
        1.48902652],
       [0.71948373, 0.71794438, 0.7326426 , ..., 1.60433237, 1.56077749,
        1.52126611],
       [0.70155918, 0.70693449, 0.70755728, ..., 1.4926572 , 1.491873  ,
        1.47145835],
       ...,
       [0.52919732, 0.54734559, 0.5840774 , ..., 0.84824407, 0.77838312,
        0.72360755],
       [0.53858705, 0.57251296, 0.55777176, ..., 1.10305929, 0.98044528,
        0.84478498],
       [0.55156547, 0.49564388, 0.82268614, ..., 1.24376358, 1.18802766,
        0.95680786]])

<IPython.core.display.Javascript object>

In [132]:
t2m_crps_baseline[0].mean()

0.9354666921537256

<IPython.core.display.Javascript object>

In [140]:
X_train_t2m_denormed[6].isel(mean_std=0, forecast_date=0).values

array([[278.7597 , 278.7826 , 278.68338, ..., 269.31384, 269.7192 ,
        270.0631 ],
       [279.01962, 279.006  , 278.9756 , ..., 269.0591 , 269.37448,
        269.71112],
       [278.90833, 278.94598, 279.03442, ..., 269.16586, 269.68805,
        270.139  ],
       ...,
       [291.6268 , 291.79593, 291.7045 , ..., 290.59763, 290.8133 ,
        291.00937],
       [291.89563, 291.32465, 290.8619 , ..., 290.1187 , 290.42224,
        290.66406],
       [291.46652, 290.55984, 289.8945 , ..., 289.108  , 289.59048,
        290.01157]], dtype=float32)

<IPython.core.display.Javascript object>

In [116]:
y_train_t2m_denormed[1]

<xarray.DataArray 't2m_truth' (forecast_date: 1429, lat: 120, lon: 130)>
array([[[277.27625, 277.56528, ..., 272.6497 , 272.87097],
        [277.1746 , 277.5274 , ..., 272.92078, 273.07428],
        ...,
        [291.52667, 291.128  , ..., 289.68082, 289.63098],
        [291.64627, 290.7293 , ..., 290.30075, 290.20508]],

       [[278.83704, 278.85098, ..., 271.1208 , 271.5972 ],
        [278.9387 , 278.92673, ..., 271.66898, 272.00586],
        ...,
        [291.7818 , 291.46286, ..., 289.54926, 289.48148],
        [291.92334, 291.10608, ..., 289.9599 , 289.7805 ]],

       ...,

       [[278.08954, 278.0733 , ..., 264.3376 , 263.81622],
        [278.66937, 278.55893, ..., 264.0745 , 264.15082],
        ...,
        [293.24805, 292.74942, ..., 289.4507 , 289.4442 ],
        [292.87775, 291.64984, ..., 290.01755, 289.9672 ]],

       [[279.2476 , 279.2476 , ..., 270.1538 , 270.31458],
        [279.6504 , 279.60977, ..., 269.82245, 269.90042],
        ...,
        [291.24542, 290.80365, ..., 289.38736, 289.6586 ],
        [291.10574, 290.06464, ..., 289.579  , 289.81778]]], dtype=float32)
Dimensions without coordinates: forecast_date, lat, lon

<IPython.core.display.Javascript object>

### 3. Train EMOS_global model for each lead_time and variable

In [72]:
def EMOS_global_train(
    var_num, lead_time, batch_size=4096, epochs=5, lr=0.1, validation_split=0.2
):
    """
    train all global EMOS models for a specific variable and lead_time
Args: 
    var_num (integer): number between 0 - 5 for each of the variables["u10", "v10", "t2m", "t850", "z500", "ws10"]
    lead_time (integer): number between 0 - 31 for each lead_time
    """
    # 0. Set up
    lead_time = lead_time + 1
    var_names = ["u10", "v10", "t2m", "t850", "z500", "ws10"]
    # 1. Load train data
    train_var_denormed = ldpd.load_data_all_train_proc_denorm()[var_num]
    # 2. Split Data X, y
    X_train_var_denormed = train_var_denormed[
        list(train_var_denormed.data_vars.keys())[0]
    ].isel(lead_time=lead_time)
    y_train_var_denormed = train_var_denormed[
        list(train_var_denormed.data_vars.keys())[1]
    ].isel(lead_time=lead_time)
    # 3. Defining cost function
    if var_num in [5]:
        crps = crps_cost_function_trunc
    else:
        crps = crps_cost_function
    # 4. Train model
    EMOS_glob = build_EMOS_network_keras(compile=True, lr=lr, loss=crps)
    EMOS_glob.fit(
        [
            X_train_var_denormed.isel(mean_std=0).values.flatten(),
            X_train_var_denormed.isel(mean_std=1).values.flatten(),
        ],
        y_train_var_denormed.values.flatten(),
        batch_size=batch_size,
        epochs=epochs,
        validation_split=validation_split,
    )
    EMOS_glob.save(
        "/home/dchen/BA_CH_EN/models/EMOS_global_models/denormed/EMOS_glob_"
        + var_names[var_num]
        + "_lead_time_"
        + str(lead_time - 1)
        + "_denormed.h5"
    )
    return EMOS_glob

<IPython.core.display.Javascript object>

In [77]:
def EMOS_global_predict_evaluate(EMOS_glob, var_num, lead_time):
    """
    Use the given EMOS_global model to predict and evaluate on a test dataset
    """
    # 0. Set up
    lead_time = lead_time + 1
    var_names = ["u10", "v10", "t2m", "t850", "z500", "ws10"]
    # 1. Load test data
    test_var_denormed = ldpd.load_data_all_test_proc_denorm()[var_num]

    # 2. Split test data
    X_test_var_denormed = test_var_denormed[
        list(test_var_denormed.data_vars.keys())[0]
    ].isel(lead_time=lead_time)

    y_test_var_denormed = test_var_denormed[
        list(test_var_denormed.data_vars.keys())[1]
    ].isel(lead_time=lead_time)

    # 3. Defining cost function
    if var_num in [5]:
        crps = crps_trunc
    else:
        crps = crps_normal

    # 4. Predict
    EMOS_glob_preds = EMOS_glob.predict(
        [
            X_test_var_denormed.isel(mean_std=0).values.flatten(),
            X_test_var_denormed.isel(mean_std=1).values.flatten(),
        ],
        verbose=1,
    )
    # 5. Evaluate based on CRPS
    EMOS_glob_crps = crps(
        mu=EMOS_glob_preds[:, 0],
        sigma=EMOS_glob_preds[:, 1],
        y=y_test_var_denormed.values.flatten(),
    )
    EMOS_glob_crps = EMOS_glob_crps.reshape(y_test_var_denormed.shape).mean(axis=0)
    return (
        EMOS_glob_crps,
        EMOS_glob_preds.reshape(y_test_var_denormed.shape[0], 120, 130, 2),
    )

<IPython.core.display.Javascript object>

In [78]:
def run_all(
    var_num, lead_time, batch_size=4096, epochs=5, lr=0.1, validation_split=0.2
):
    EMOS_glob = EMOS_global_train(
        var_num,
        lead_time,
        batch_size=batch_size,
        epochs=epochs,
        lr=lr,
        validation_split=validation_split,
    )
    return EMOS_global_predict_evaluate(EMOS_glob, var_num, lead_time)

<IPython.core.display.Javascript object>

In [79]:
EMOS_glob_t2m_0_scores, EMOS_glob_t2m_0_preds = run_all(2, 0, epochs=1)

174038/174038 [==============================] - 108s 619us/step


<IPython.core.display.Javascript object>

In [71]:
EMOS_glob_t2m_0_scores.mean()

2404.599984658536

<IPython.core.display.Javascript object>

In [ ]:
crps_normal()

In [90]:
EMOS_glob_t2m_0_preds.shape

(357, 120, 130, 2)

<IPython.core.display.Javascript object>

In [99]:
y_test_t2m_denormed[1]

<xarray.DataArray 't2m_test_truth' (forecast_date: 357, lat: 120, lon: 130)>
array([[[277.0977 , 277.12692, ..., 261.9615 , 261.7991 ],
        [278.06055, 278.0573 , ..., 263.94565, 263.5235 ],
        ...,
        [293.34613, 293.37375, ..., 290.06137, 290.28708],
        [292.9792 , 292.5083 , ..., 289.96396, 290.25623]],

       [[274.01102, 273.90225, ..., 268.82816, 268.98566],
        [274.47217, 274.39423, ..., 268.682  , 268.77133],
        ...,
        [293.65466, 293.68713, ..., 290.1312 , 290.4657 ],
        [293.79428, 293.5686 , ..., 289.91364, 290.31955]],

       ...,

       [[276.01627, 276.0374 , ..., 264.49118, 264.82986],
        [276.324  , 276.33865, ..., 264.8966 , 265.36557],
        ...,
        [292.77957, 292.59393, ..., 291.99637, 291.9703 ],
        [292.3888 , 291.55673, ..., 291.70978, 291.92636]],

       [[274.96274, 274.92694, ..., 261.42346, 261.7703 ],
        [275.56033, 275.56522, ..., 259.2513 , 259.7105 ],
        ...,
        [292.88705, 292.67212, ..., 291.90845, 291.98172],
        [292.41483, 291.43948, ..., 291.68698, 291.91333]]], dtype=float32)
Dimensions without coordinates: forecast_date, lat, lon

<IPython.core.display.Javascript object>

In [101]:
EMOS_glob_t2m_0_preds[:, :, :, 0]

array([[[-2391.4802, -2392.1772, -2391.4294, ..., -2262.4302,
         -2262.737 , -2263.2532],
        [-2396.235 , -2396.2266, -2395.0044, ..., -2266.2212,
         -2265.9072, -2265.4917],
        [-2399.7861, -2400.1763, -2400.6453, ..., -2260.3098,
         -2261.4966, -2261.3464],
        ...,
        [-2511.708 , -2512.699 , -2512.696 , ..., -2476.719 ,
         -2478.7607, -2479.4866],
        [-2512.2537, -2508.2964, -2507.1194, ..., -2474.8147,
         -2477.214 , -2478.4998],
        [-2506.028 , -2500.    , -2496.793 , ..., -2467.4924,
         -2471.8716, -2475.9543]],

       [[-2351.2478, -2351.9917, -2351.4155, ..., -2291.2268,
         -2291.2742, -2289.9065],
        [-2355.7397, -2356.6836, -2357.107 , ..., -2290.7034,
         -2290.754 , -2289.1323],
        [-2364.1782, -2364.644 , -2364.6165, ..., -2295.588 ,
         -2295.878 , -2293.7036],
        ...,
        [-2506.4954, -2506.6182, -2505.1384, ..., -2480.4473,
         -2482.9646, -2484.1106],
        [-25

<IPython.core.display.Javascript object>